In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter

import warnings
warnings.filterwarnings("ignore")


In [2]:
def rmse(predictions, targets):
    return np.sqrt(np.mean((predictions-targets)**2))

In [22]:
def model(x, target):
    #split data
    X_train, X_test, y_train, y_test = train_test_split(x, df[target], random_state=42)
    
    #create model
    model = LinearRegression()
    #fit data to model
    model.fit(X_train, y_train)
    
    #Make predictions
    train_pred = model.predict(X_train)
    test_pred = model.predict(X_test)
    
    #test rmse
    test_rmse = rmse(test_pred, y_test.values)
    return round(test_rmse, 2)

# import data

In [4]:
path = "../../../data/kc_house_data.csv"

In [5]:
df = pd.read_csv(path)

In [6]:
df.head(2)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,10/13/2014,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,12/9/2014,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639


# Clean the data

### turn columns into floats

In [7]:
df['sqft_living'] = df['sqft_living'].astype(float)
df['bedrooms'] = df['bedrooms'].astype(float)
df['condition'] = df['condition'].astype(float)
df['grade'] = df['grade'].astype(float)
df['sqft_above'] = df['sqft_above'].astype(float)
df['yr_built'] = df['yr_built'].astype(float)
df['zipcode'] = df['zipcode'].astype(float)
df['sqft_living15'] = df['sqft_living15'].astype(float)
df['sqft_lot15'] = df['sqft_lot15'].astype(float)
df['sqft_lot'] = df['sqft_lot'].astype(float)

### Drop useless columns

In [8]:
df.drop(["waterfront", "yr_renovated", "view", "lat"], axis=1, inplace=True)

### Drop na from dataset

In [9]:
df.dropna(inplace=True)

# Create new column

In [10]:
df["age"] = df["yr_built"].map(lambda x: 2021 - x)

# Counter of how many times each unique element appears in categorical columns

In [11]:
Counter(df["zipcode"])

Counter({98178.0: 262,
         98125.0: 409,
         98028.0: 283,
         98136.0: 263,
         98074.0: 441,
         98053.0: 403,
         98003.0: 280,
         98198.0: 280,
         98146.0: 288,
         98038.0: 589,
         98007.0: 141,
         98115.0: 583,
         98107.0: 266,
         98126.0: 354,
         98019.0: 190,
         98103.0: 602,
         98002.0: 199,
         98133.0: 493,
         98040.0: 282,
         98092.0: 351,
         98030.0: 256,
         98119.0: 184,
         98112.0: 269,
         98052.0: 574,
         98027.0: 412,
         98117.0: 553,
         98058.0: 455,
         98001.0: 361,
         98056.0: 406,
         98166.0: 254,
         98023.0: 499,
         98070.0: 117,
         98148.0: 57,
         98105.0: 229,
         98042.0: 547,
         98008.0: 283,
         98059.0: 468,
         98122.0: 290,
         98144.0: 343,
         98004.0: 317,
         98005.0: 168,
         98034.0: 545,
         98075.0: 359,
         981

# Get ascending list of correlations

In [12]:
corrs = df.corr()["price"].sort_values(ascending=False).drop("price")

# Get top cross correlations

In [13]:
def top_entries(df):
    #df.corr().abs().unstack().sort_values(ascending=False).drop_duplicates()
    
    mat = df.corr().abs()
    
    # Remove duplicate and identity entries
    mat.loc[:,:] = np.tril(mat.values, k=-1)
    mat = mat[mat>0]

    # Unstack, sort ascending, and reset the index, so features are in columns
    # instead of indexes (allowing e.g. a pretty print in Jupyter).
    # Also rename these it for good measure.
    return (mat.unstack()
             .sort_values(ascending=False)
             .reset_index()
             .rename(columns={
                 "level_0": "feature_a",
                 "level_1": "feature_b",
                 0: "correlation"
             }))

In [14]:
top_entries(df).dropna().head(10)

,feature_a,feature_b,correlation
0,yr_built,age,1.000000
1,sqft_living,sqft_above,0.876448
2,sqft_living,grade,0.762779
3,sqft_living,sqft_living15,0.756402
4,grade,sqft_above,0.756073
5,bathrooms,sqft_living,0.755758
6,sqft_above,sqft_living15,0.731767
7,sqft_lot,sqft_lot15,0.718204
8,grade,sqft_living15,0.713867
9,price,sqft_living,0.701917


# Create variable called target that contains price column of full df

In [15]:
target = "price"

# Create baseline model

In [23]:
baseline_df = df[[corrs.keys()[0]]]

In [24]:
model(baseline_df, target)

262216.52

# Create model with top 3

In [25]:
top3_df = df[[corrs.keys()[0], corrs.keys()[1], corrs.keys()[2]]]

In [26]:
model(top3_df, target)

253195.5

# Top 7 correlated

In [27]:
top7_df = df[[corrs.keys()[0], corrs.keys()[1], corrs.keys()[2], corrs.keys()[3], corrs.keys()[4], corrs.keys()[5], corrs.keys()[6]]]

In [28]:
model(top7_df, target)

250430.9